## Connect to ICW:

In [ ]:
import cadspy
import numpy as np
import pandas as pd
from datetime import datetime as dt
from itertools import combinations

# diplay all rows and cols when using 'dataframe'.head() or 'dataframe'.tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [ ]:
icw = cadspy.DatabaseConnection(system='ICW', user='u204570')

<br>

### Data

#### S19 Lounge Eligibility data

In [ ]:
# lounge eligibility data
query = """

sel * from LDB_SBOX_OR.HACKATHON_OPS_LOUNGE_ELIGIBILITY

"""

df_lounge_eligibility = icw.queryToDataframe(query)

# flight info data
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_FLIGHT_INFO

"""

df_flight_info = icw.queryToDataframe(query)

# country decode data
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_COUNTRY_DECODE

"""

df_country = icw.queryToDataframe(query)

# AC_type
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_AC_TYPE

"""

df_acft_typ = icw.queryToDataframe(query)

<br>

### Pre-processing

*Hint:* It is always worth checking the format of each of the columns in your dataframes before trying to do any work with them. To do so, you can make use of the `headers_and_first_row` function below.


In [ ]:
def headers_and_first_row(df):
    '''
    print headers and first row of a df to deal with data types
    '''
    
    headers = df.columns
    first_row = []

    for col in headers:
        first_row.append(df[col][0])
    
    dictionary = dict( zip( headers, first_row) )

    return dictionary

# helper function

def dataframe_str_formatter(df):
    '''Strips all whitespace in string columns in DataFrame'''
    for col in df.columns:
        if isinstance(df[col][0],str):
            df[col] = df[col].str.strip()
        else:
            continue
    return df


In [ ]:
# applying headers_and_first_row to df_lounge_eligibility
format_df = headers_and_first_row(df_lounge_eligibility)

# Note that some columns have blank spaces!
format_df

In [ ]:
# pre-processing code

df_lounge_eligibility = dataframe_str_formatter(df_lounge_eligibility)

In [ ]:
# pre-processing code

df_lounge_eligibility = dataframe_str_formatter(df_lounge_eligibility)
df_lounge_eligibility['GMT_UPLIFT_DT'] = pd.to_datetime(df_lounge_eligibility['GMT_UPLIFT_DT'])


<br><br>

### df_flight_info pre-processing

In [ ]:
format_df = headers_and_first_row(df_flight_info)

format_df

In [ ]:
# pre-processing code

df_flight_info = dataframe_str_formatter(df_flight_info)

# Need to convert to same date type for merge
df_flight_info['GMT_PLND_DEP_TS'] = pd.to_datetime(df_flight_info['GMT_PLND_DEP_TS'], format = '%Y-%m-%d')
df_flight_info['GMT_PLND_DEP'] = pd.to_datetime(df_flight_info['GMT_PLND_DEP_TS'].dt.date)

format_df = headers_and_first_row(df_flight_info)

format_df

### df_country pre-processing

In [ ]:
# show current format

format_df_country = headers_and_first_row(df_country)

format_df_country

# pre-processing code

df_country = dataframe_str_formatter(df_country)

format_df_country = headers_and_first_row(df_country)

format_df_country

### df_acft_type pre-processing

In [ ]:
# show current format

format_df_aircraft = headers_and_first_row(df_acft_typ)

format_df_aircraft

# pre-processing code

df_acft_type = dataframe_str_formatter(df_acft_typ)

format_df_aircraft = headers_and_first_row(df_acft_typ)

format_df_aircraft

In [ ]:
# Example

# joining df_lounge_eligibility and df_flight_info

df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['GMT_UPLIFT_DT','OPERATING_FLT_NO','UPLIFT_STN_CD','DISCHARGE_STN_CD'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['GMT_PLND_DEP','OPG_FLT_NO','ACT_DEP_STN_CD','ACT_ARR_STN_CD'], # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

print('Old Shape: {}'.format(df_lounge_eligibility.shape))
print('New Shape: {}'.format(df_lounge_elig_flight_info.shape))

### Check Join has worked correctly by looking at some rows and countings null where join may not have worked

In [ ]:
# your code here!

df_lounge_country_flight = pd.merge(df_lounge_elig_flight_info,
                                   df_country,
                                   on = 'ROUTE',
                                   how = 'left'
                                   )

print('Old Shape: {}'.format(df_lounge_elig_flight_info.shape))
print('New Shape: {}'.format(df_lounge_country_flight.shape))

In [ ]:
df_final = pd.merge(df_lounge_country_flight,
                   df_acft_typ,
                   on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
                   how = 'left'

                   )

print('Old Shape: {}'.format(df_lounge_country_flight.shape))
print('New Shape: {}'.format(df_final.shape))

<br><br><br>

In [ ]:
#--------------------------------
#
# Your turn!!!
#
#--------------------------------

<u>What level of granularity do you use?</u>

This project wants to understand what characteristics of a flight help us get a better picture of the lounge eligibility profiles.
Things I want to consider:
- Time of flight (Morning, Afternoon, Evening) or maybe even by hour
- Destination as this determines the passenger profile, different countries/regions have more premium passengers potentially
- Short Haul/Medium Haul/Long Haul 
- Month of Flight
- City Route Flag
- Weekday/Weekend 
- Business Route indicator?


<u>General Thoughts:</u>

Destination:
- Routes are too granular as if we have new routes in the future we wouldn't be able to estimate
- Countries could also be granular if we develop new routes to countries
- Region may be suited as we currently fly to all regions
- We need to think how to consider how certain countries within a region may have higher premium loads vs others

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- ... (your answer here)

## Data Exploration

In [ ]:
df_final.head()

In [ ]:
def tier_proportions(features, df_fin):
    
    '''A function to build our the proportion of customers eligible for lounge based on features inputted'''

    tier_col = 'Lounge_eligibility_tier'
    
    # Create group by parameters
    groupby_list = features[:]
    groupby_list.append(tier_col)

    # Get total eligible by features and tier
    df_grouped = df_fin.groupby(groupby_list).agg({'pax':'sum'})

    # Add total amount for features excluding tier to get breakdown
    df_grouped['total_pax_features'] = df_grouped.groupby(features)['pax'].transform('sum')

    # proportion elgible 
    df_grouped['proportion'] = (df_grouped['pax'] / df_grouped['total_pax_features']) * 100
    df_grouped['proportion'] = df_grouped['proportion'].map('{:,.1f}%'.format)

    # reset index
    df_grouped.reset_index(inplace = True)

    # remove not eligible passengers
    df_grouped = df_grouped[df_grouped[tier_col] != 'Not eligible']

    # reshape data required for Ops Team
    df = df_grouped.pivot(index = features, columns = tier_col, values = 'proportion')
    
    return df


<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

## Evaluation

In [ ]:
# Build out the daily passengers for each flight

def feature_pax_2019(features, df):
    
    '''Computes the passengers who travelled by features and flight number and day'''
    
    eval_list = features[:]
    group_features = ['GMT_UPLIFT_DT','OPERATING_FLT_NO']
    prop_features = group_features[:]
    prop_features.extend(eval_list)
    df_flight_sum = df_final.groupby(prop_features)['pax'].sum().reset_index()
    return df_flight_sum, group_features
    
def proportion_formatter(df):
    
    '''Simple function to reformat the tier eligibility table for evaluation joins'''
    
    # format the proportions for the join
    df_proportions = df.stack().reset_index()

    # rename proportion column
    df_proportions = df_proportions.rename({0:'Proportions'}, axis = 1)
    
    return df_proportions

def flight_tier_forecast(df_prop, df_feat, feat_list):
    
    '''Forecasts the number of passengers in each tier for each flight'''
    
    # join the two

    df_total = pd.merge(df_feat, df_prop, how = 'inner', on = feat_list)

    # calculate proportion of each tier in each flight
    df_total['Proportion Numeric'] = df_total['Proportions'].str.replace('%','').astype(float) / 100
    df_total['tier_pax'] = df_total['pax'] * df_total['Proportion Numeric']
    df_total['tier_pax'] = df_total['tier_pax'].values.round()

    return df_total

def eval_table(df_final, df_tot, group_features):

    '''Function to create the evaluation table which compares forecast vs actual'''
    
    tier = 'Lounge_eligibility_tier'

    tier_features = group_features[:]
    tier_features.append(tier)

    df_tier = df_final.groupby(by = tier_features)['pax'].sum().reset_index()
    df_eval = pd.merge(df_tier, df_tot, how = 'inner', on = tier_features)
    
    return  df_eval

def rmse(df_eval):
    
    '''Returns the Root Mean Squared Error of our prediction'''
    
    df_eval['sqrd_resid'] = (df_eval['pax_x'] - df_eval['tier_pax']) ** 2
    std = np.sqrt(df_eval['sqrd_resid'].sum() / df_eval.shape[0])
    
    return std

def evaluation(feat, df_fin, df_test):
    
    '''Function which returns the RMSE of the forecast'''
    
    df_feat, grp_feat = feature_pax_2019(feat, df_fin)
    df_prop_feat = proportion_formatter(df_test)
    df_total = flight_tier_forecast(df_prop_feat, df_feat, feat)
    df_eval = eval_table(df_fin, df_total, grp_feat)
    std = rmse(df_eval)
    return std
    
    

## Feature Evaluation

In [ ]:
def feature_evaluation(feat, df):
    
    df_test = tier_proportions(feat, df)
    result = evaluation(feat, df, df_test)
    return result
    
def feature_combination(features):
    
    '''Simple function which returns every possible combination of features'''
    
    nbr_features = len(features)
    feature_comb_list = []
    for nbr in range(1,nbr_features + 1):
        stage_comb = [list(comb) for comb in combinations(features, nbr)]
        feature_comb_list.extend(stage_comb)
        
    return feature_comb_list
    
def combination_evaluation(features, df):
    
    '''A function to test every combination and evaluate its performance'''
    
    results = []
    test_features = feature_combination(features)
    
    # loop through every combination of features and test result
    for feat_comb in test_features:
        rmse = feature_evaluation(feat_comb, df)
        results.append([feat_comb,rmse])
        
    # create dataframe of results
    df_result = pd.DataFrame(results, columns = ['feature_combination','RMSE'])
    df_result = df_result.sort_values('RMSE', ascending = True)
    
    return df_result

In [ ]:
features = ['CORP_GEOG_CTRY_GRP_NM_x', 'WB_NB_CAT']

In [ ]:
final_table = combination_evaluation(features, df_final)
final_table